In [32]:
import os
import sys
import random
import urllib.parse
import time

import markovify
import json
from bs4 import BeautifulSoup

sys.path.insert(0, os.path.abspath('..'))

import ao3
from ao3.utils import work_id_from_url
from ao3.works import Work

In [2]:
def add_work_to_chain(work, models, state_size=3):
    '''adds new Markov chain model of work
    with given ID to the dict models and returns said dict'''
    text = work.get_work_text()
    model = markovify.Text(text, state_size=state_size)
    models[str(work.id)] = model
    print('Added work {} to models.'.format(work.title))

    return models

def combine_models(models, weights=None):
    if not weights:
        model = markovify.combine(list(models.items()))
    else:
        model = markovify.combine(list(models.items()), weights)
    return model.compile()

In [3]:
def load_data():
    with open('data.txt', 'r') as f:
        raw_data = f.readlines()

    bookmark_data = {}
    in_bookmark = False
    ending_bookmark = False
    current_id = '3413384'
    current_data = ''
    for line in raw_data:
        if ending_bookmark:
            # print('Bookmark is ending')
            ending_bookmark = False
            parts = line.partition('>')
            current_data += parts[0]
            current_data += parts[1]
            # print('Partitioned line: {}'.format(parts))
            bookmark_data[current_id] = current_data # add tag to dict
            # print('Assigned current data to id {}'.format(current_id))
            current_id = parts[2].strip() # assign next id
            # print('Assigned next id as {}'.format(current_id))
            current_data = ''
        else:
            current_data += line
            if '<div class="recent dynamic"' in line:
                ending_bookmark = True
                # print('Turned on ending bookmark')
    
    print('Logging in...')
    api.login('starrybouquet')
    all_works = []
    for id, html in bookmark_data.items():
        soup = BeautifulSoup(html, 'html.parser')
        all_works.append(Work(id, api.handler, load=False, soup=soup))
        
    possible_tags = set(['Samantha "Sam" Carter/Jack O\'Neill', 'Jack/Sam', 'Sam/Jack', 'Sam Carter/Jack O\'Neill'])
    matching_works = []
    for work in all_works:
        ship = set(work.relationship)
        intersection = possible_tags.intersection(ship)
        if len(intersection) > 0:
            matching_works.append(work)
    print('Works matching tags: ')
    print(len(matching_works))
    return matching_works

In [4]:
def create_models(works):
    models = {} # each entry is id: model
    ids = []
    
    models_built = 1
    for work in works:
        print('Building model {}...'.format(models_built))
        if models_built % 40 == 0:
            print('{} models built, sleeping 3 min'.format(models_built))
            time.sleep(180)
        models = add_work_to_chain(work, models)
        models_built += 1
    print('Done building models.')
    print('keys of model dict:')
    print(models.keys())
    return models

In [38]:
def combine_json_models():
    # load
    with open('work_models.json', 'r') as f:
        models_from_json = json.loads(f.read())
    modelList = []
    models = {}
    for workid, model_json in models_from_json.items():
        actual_model = markovify.Text.from_json(model_json)
        modelList.append(actual_model)
        models[str(workid)] = actual_model

    # combine
    full_model = combine_models(modelList)
    num_sentences = int(input('Enter # of sentences to try: '))
    for i in range(num_sentences):
        print(full_model.make_sentence())

    save = str(input('Do you want to save this model? y or n: '))
    if save == 'y':
        filename = str(input('Enter filename to save: '))
        json_model = full_model.to_json()
        with open(filename, 'w') as f:
            json.dump(json_model, f)
        print('Saved file. IDs used were:')
        print(ids)

In [7]:
api = ao3.AO3()
works = load_data()
input('Press Enter to continue: ')
models = create_models(works)

Logging in...
Password: ········
Authentication successful. You are logged in.
Works matching tags: 
179
Press Enter to continue: 
Building model 1...
Added work Auctions and Consequences to models.
Building model 2...
Added work Better or Right to models.
Building model 3...
Added work dark switch to models.
Building model 4...
Added work level three, advanced to models.
Building model 5...
Added work If the World was Ending to models.
Building model 6...
Added work Walking the Dog to models.
Building model 7...
Added work a shine of light and a circle of blue to models.
Building model 8...
Added work F*@kingPete to models.
Building model 9...
Added work T-minus a Lifetime to models.
Building model 10...
Added work Walking on Broken Glass to models.
Building model 11...
Added work Singing to models.
Building model 12...
Added work Despair and hope to models.
Building model 13...
Added work Make it rain to models.
Building model 14...
Added work Revelations to models.
Building model 15

Added work The Perfect Gift to models.
Building model 132...
Added work Birthday Surprise to models.
Building model 133...
Added work Five Pieces of Furniture to models.
Building model 134...
Added work Rules of Engagement to models.
Building model 135...
Added work Life Lessons to models.
Building model 136...
Added work Distant Stars to models.
Building model 137...
Added work Together to models.
Building model 138...
Added work School's Out to models.
Building model 139...
Added work The Consequences of Forgetting Beer to models.
Building model 140...
Added work The unexpected kisses to models.
Building model 141...
Added work Smaller Victories to models.
Building model 142...
Added work Can't Fight This Feeling to models.
Building model 143...
Added work Soldier On to models.
Building model 144...
Added work Forget I Even Asked You to models.
Building model 145...
Added work First Times and Second Chances to models.
Building model 146...
Added work Blind Date to models.
Building mo

In [29]:
json_models = {}
models_list = []
for workid, model in models.items():
    json_model = model.to_json()
    json_models[workid] = json_model


In [34]:
with open('work_models.json', 'w') as file:
    json.dump(json_models,file)

In [39]:
combine_json_models()

AttributeError: 'list' object has no attribute 'items'